In [77]:
import xml.etree.ElementTree as ET 
import os

In [78]:
#Input file name, change this variable for a different file
inputFilename = 'agh-fis-spr17.xml'

# Passing the path of the 
# xml document to enable the 
# parsing process 
tree = ET.parse(inputFilename) 

#getting just the filename without extension for output file name
trimmedName = os.path.splitext(inputFilename)[0]

In [79]:
# getting the parent tag of 
# the xml document 
root = tree.getroot() 

In [80]:
#Find and remove rooms tag
rooms = root.find('rooms')
root.remove(rooms)

In [81]:
#Find optimization tag
optimization = root.find('optimization')

In [82]:
#remove room atribute in optimization
optimization.attrib.pop('room')

'1'

In [83]:
#Find courses tags
courses = root.find('courses')

In [84]:
#Getting all classes from all courses
classes = courses.findall('./course/config/subpart/class')

In [85]:
#Removig all room children from each class parent
for Class in classes:
    if(Class.get('room') != None):
        Class.attrib.pop('room')
    rooms = Class.findall('./room')
    for room in rooms:
        Class.remove(room)

In [86]:
distRoot = root.find('distributions')

#Finding and removing same-room constraints
distributionsXML = distRoot.findall('./distribution')

#Listing the required types of constraints to be extracted
softTypes = ['SameTime', 'DifferentTime', 'MinGap', 'DifferentDays']
hardTypes = ['Precedence', 'SameAttendees', 'NotOverlap']

#Iterating through all distribution constraints
for dist in distributionsXML:
    #getting type of constraints to be checked
    constraintType = dist.get('type')
    #Removing the () from MinGap(G) for simpler comparison
    constraintType = constraintType.split('(')[0]
    #Setting default value to False for keeping current constraint
    keep = False
    #Checking if the current type is in Soft constraint types
    if constraintType in softTypes:
        #Checking if the given constraint is soft (all soft constriants will have penalty value)
        if dist.get('penalty') is not None:
            keep = True
    #Checking if the current type is in Hard constraint types
    elif constraintType in hardTypes:
        #Checking if the given constraint is hard (all hard constriants will have reqired as true)
        if(dist.get('required') == 'true'):
            keep = True

    #If the current constraint is not required, the keep value is never changed to True and hence removing the same from constraints
    if keep is False:
        distRoot.remove(dist)
        
        

In [87]:
outputFilename = trimmedName + '_trimmed.xml'
tree.write(outputFilename)

In [88]:
#Copying initial lines from input file
with open(inputFilename) as myfile:
    head = [next(myfile) for x in range(1)]

with open(outputFilename, 'r') as trimmed:
    content = trimmed.read() 


with open(outputFilename, 'w') as trimmed:
    trimmed.seek(0,0)
    trimmed.write(head[0]+'\n'+content)